### NOTEBOOKS PARA REALIZAR PRUEBAS Y ANÁLISIS EXPLORATORIO

A continuación se muestra el proceso desde el inicio para construir un buscador en base al corpus de reuters, se realizará cada paso:
1. Construcción de un archivo .csv que contenga todos los textos de reuters unificados (corpus).
2. Implementación de funciones para preprocesar y normalizar cada texto de reuters.
3. Construcción de un nuevo archivo .csv con todos los textos de nuestro corpus preprocesados y normalizados.
4. Implementación de las funciones de vectorización (TF-IDF, Bag Of Words, Word2Vec).
5. Construcción de un índice inverso que mapee términos a documentos.
6. Implementación de funciones de búsqueda que utilicen como base distancia coseno para realizar consultas y obtener los documentos más relevantes de acuerdo al método que se haya escogido (TF-IDF, BOW, Word2Vec, Índice invertido).

Se documentará cada paso del proceso, explicando que librerías son necesarias, que hace cada función y cosas relevantes. Cuando se tengan implementadas ya todas las funcionalidades destacadas anteriormente, se procederá a modularizar las funciones en varios archivos .py para que puedan ser implementados con interfaz.

Nota: Para la interfaz vamos a utilizar Flask, con sus respectivas rutas y demás.

In [67]:
import pandas as pd
import os
import spacy
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict


directorio_base = os.path.join(os.getcwd(), "..")  # Volver a directorio RI_PROYECTO_1B

In [ ]:
# Ejecutar una vez para tener las dependencias necesarias de nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

In [ ]:
# Ejecutar una vez para obtener dependencias necesarias
%pip install scikit-learn gensim

### 1. Construcción del corpus

A partir de todos los archivos de reuters, vamos a construir un solo archivo .csv, en el cual vamos a tener todos estos textos, para este fin vamos a realizar el siguiente proceso:
1. Iterar sobre las carpetas /data/test y /data/training
2. Iterar sobre cada archivo dentro de esas carpetas
3. Obtener el contenido de dicho archivo
4. Guardar en una lista [id_archivo, contenido_archivio, path_archivo]

Cuando se finalicen todas las iteraciones, esta lista va a ser convertida en un DataFrame de pandas y, posteriormente a un archivo .csv con sus respectivas 3 columnas mencionadas anteriormente.


In [ ]:
subcarpeta_reuters = [f"{directorio_base}\\data\\training",
                      f"{directorio_base}\\data\\test"]

noticias = []

for subcarpeta in subcarpeta_reuters:
    textos = os.scandir(subcarpeta)

    for texto in textos:
        with open(texto, 'r', encoding='utf-8', errors='ignore') as file:
            nombre_archivo = file.name.split("\\")[-1]
            subcarpeta_texto = subcarpeta.split("\\")[-1]
            print(f"Iterando sobre texto {nombre_archivo} en subcarpeta {subcarpeta_texto}")
            contenido = file.read()

            noticias.append([nombre_archivo, contenido, subcarpeta_texto])

Iterando sobre texto 1 en subcarpeta training
Iterando sobre texto 10 en subcarpeta training
Iterando sobre texto 100 en subcarpeta training
Iterando sobre texto 1000 en subcarpeta training
Iterando sobre texto 10000 en subcarpeta training
Iterando sobre texto 10002 en subcarpeta training
Iterando sobre texto 10005 en subcarpeta training
Iterando sobre texto 10008 en subcarpeta training
Iterando sobre texto 10011 en subcarpeta training
Iterando sobre texto 10014 en subcarpeta training
Iterando sobre texto 10015 en subcarpeta training
Iterando sobre texto 10018 en subcarpeta training
Iterando sobre texto 10023 en subcarpeta training
Iterando sobre texto 10025 en subcarpeta training
Iterando sobre texto 10027 en subcarpeta training
Iterando sobre texto 1003 en subcarpeta training
Iterando sobre texto 10032 en subcarpeta training
Iterando sobre texto 10035 en subcarpeta training
Iterando sobre texto 10037 en subcarpeta training
Iterando sobre texto 10038 en subcarpeta training
Iterando so

In [32]:
df_textos_corpus = pd.DataFrame(noticias, columns=["id", "contenido", "subcarpeta"])

In [ ]:
df_textos_corpus.to_csv(f"{directorio_base}\\data\\corpus_sin_procesar.csv", index=False)

### 2. Preprocesar los corpus obtenidos

Hemos obtenido los textos que van a conformar nuestro corpus y los hemos concatenado todos en un archivo .csv el cual podemos manipularlo de una manera mucho más fácil y eficiente. Ahora lo que vamos a realizar es el preprocesamiento de este corpus, para ello vamos a realizar el siguiente proceso:

1. Extracción de contenido relevante de los documentos
2. Eliminar caracteres no deseados y normalizar el texto
3. Tokenización del texto
4. Eliminación de stop words y aplicación de lematización

Para este apartado vamos a crear una sola función de nombre preprocesar_contenido, la cual va a realizar todo el proceso de tokenización y normalización de cada contenido de cada documento que conforma el corpus

In [54]:
df_corpus_sp = pd.read_csv(f"{directorio_base}\data\corpus_sin_procesar.csv")

In [88]:
# Obtencion de las stopwords desde el archivo 'stopwords'
 
with open(f"{directorio_base}\data\stopwords", mode='r', encoding='utf-8') as file:
    contenido_stopwords = file.read()

stopwords = []

for words in contenido_stopwords.split("\n"):
    stopword = words.replace("\n", "")
    stopwords.append(stopword)

print(stopwords)

['a', "a's", 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'conta

In [56]:
signos_puntuacion = [",", ".", ":", "...", "-", "_", "+", ";", '"', "(", ")", "[", "]", "%", "$", "#", "@", "&", "?", "!", "/", ">", "<"]

In [57]:
# Configuración de lematizador y stopwords
lemmatizer = WordNetLemmatizer()

# Función para convertir etiquetas POS de nltk a WordNet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Por defecto

# Función de preprocesamiento
def preprocesar_contenido(contenido):
    # Convertir texto a minúsculas
    contenido = contenido.lower()

    # Reemplazar caracteres y limpiar texto
    contenido = contenido.replace("'", " ").replace(" &lt;", " ").replace("&lt;", " ")
    contenido = contenido.replace("trading", "trade").replace("united states of america", "usa").replace("united states", "usa")
    contenido = re.sub(r'\d+', '', contenido)  # Eliminar números
    contenido = re.sub(r'\s+', ' ', contenido)  # Eliminar saltos de línea y espacios redundantes

    # Remover signos de puntuación
    for signo in signos_puntuacion:
        contenido = contenido.replace(signo, "")

    # Tokenización
    contenido_tokenizado = contenido.split()

    # Eliminar stopwords
    tokens_sin_stopwords = [word for word in contenido_tokenizado if word not in stopwords]

    # Etiquetado POS (Part Of Speech)
    tokens_pos = pos_tag(tokens_sin_stopwords)
    #print(tokens_pos)

    # Lematización utilizando etiquetado POS
    tokens_lematizados = [
        lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in tokens_pos
    ]

    # Reconstruir texto lematizado a un solo string
    contenido_lematizado_str = " ".join(tokens_lematizados)

    return contenido_lematizado_str


In [58]:
print(preprocesar_contenido("Trading trading better running jumped, saw on the United States Of America is really easy teams united to win championship"))

trade trade run jump usa easy team unite win championship


In [59]:
for index, row in df_corpus_sp.iterrows():
    print(f"Iterando sobre elemento {index+1} del .csv de los corpus")
    contenido = row["contenido"]
    contenido_preprocesado = preprocesar_contenido(contenido)
    df_corpus_sp.at[index, "contenido_preproc_str"] = contenido_preprocesado

Iterando sobre elemento 1 del .csv de los corpus
Iterando sobre elemento 2 del .csv de los corpus
Iterando sobre elemento 3 del .csv de los corpus
Iterando sobre elemento 4 del .csv de los corpus
Iterando sobre elemento 5 del .csv de los corpus
Iterando sobre elemento 6 del .csv de los corpus
Iterando sobre elemento 7 del .csv de los corpus
Iterando sobre elemento 8 del .csv de los corpus
Iterando sobre elemento 9 del .csv de los corpus
Iterando sobre elemento 10 del .csv de los corpus
Iterando sobre elemento 11 del .csv de los corpus
Iterando sobre elemento 12 del .csv de los corpus
Iterando sobre elemento 13 del .csv de los corpus
Iterando sobre elemento 14 del .csv de los corpus
Iterando sobre elemento 15 del .csv de los corpus
Iterando sobre elemento 16 del .csv de los corpus
Iterando sobre elemento 17 del .csv de los corpus
Iterando sobre elemento 18 del .csv de los corpus
Iterando sobre elemento 19 del .csv de los corpus
Iterando sobre elemento 20 del .csv de los corpus
Iterando 

In [60]:
df_corpus_sp

,id,contenido,subcarpeta,contenido_preproc_str
0,1,BAHIA COCOA REVIEW\n Showers continued throug...,training,bahia cocoa review shower continue week bahia ...
1,10,COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES ...,training,computer terminal system cpml completes sale c...
2,100,N.Z. TRADING BANK DEPOSIT GROWTH RISES SLIGHTL...,training,nz trade bank deposit growth rise slightly zea...
3,1000,NATIONAL AMUSEMENTS AGAIN UPS VIACOM &lt;VIA> ...,training,national amusement ups viacom bid viacom inter...
4,10000,ROGERS &lt;ROG> SEES 1ST QTR NET UP SIGNIFICAN...,training,rogers rog see st qtr net significantly rogers...
...,...,...,...,...
10783,21571,N.Z.'S CHASE CORP MAKES OFFER FOR ENTREGROWTH\...,test,nz chase corp make offer entregrowth chase cor...
10784,21573,TOKYO DEALERS SEE DOLLAR POISED TO BREACH 140 ...,test,tokyo dealer dollar poise breach yen tokyo for...
10785,21574,JAPAN/INDIA CONFERENCE CUTS GULF WAR RISK CHAR...,test,japanindia conference cut gulf war risk charge...
10786,21575,SOVIET INDUSTRIAL GROWTH/TRADE SLOWER IN 1987\...,test,soviet industrial growthtrade slow soviet unio...


In [46]:
df_corpus_sp.to_csv(f"{directorio_base}\data\corpus_procesado.csv", index=False)

### Representación de datos en espacio vectorial:

Utilización de las siguientes técnicas:
1. Bag Of Words (BOW)
2. TF-IDF
3. Word2Vec

Vamos a aplicar estas tres técnicas de vectorización para nuestro corpus, se construirá una función para cada tipo de vectorización, la de TF-IDF y BOW devolverán el corpus vectorizado y el vectorizador usado, la de Word2Vec únicamente retornará el corpus vectorizado utilizando el modelo word2vec de google.

In [61]:
# Cargar el modelo preentrenado Word2Vec
word2vec_model = api.load("word2vec-google-news-300")

# ---- Bag Of Words ----
def vectorize_bow(corpus):
    """
    Vectoriza un corpus utilizando Bag of Words.
    """
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer

# ---- TF-IDF ----
def vectorize_tfidf(corpus):
    """
    Vectoriza un corpus utilizando TF-IDF.
    """
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer

# ---- Word2Vec ----
def vectorize_word2vec(corpus_tokens, w2v_model):
    """
    Vectoriza un corpus utilizando Word2Vec preentrenado.
    Cada documento se convierte en un vector promedio.
    """
    def get_avg_word2vec(tokens, w2v_model):
        valid_vectors = [w2v_model[word] for word in tokens if word in w2v_model]
        if valid_vectors:
            return np.mean(valid_vectors)
        else:
            return np.zeros(w2v_model.vector_size)

    vectors = [get_avg_word2vec(tokens, word2vec_model) for tokens in corpus_tokens]
    return np.vstack(vectors)


In [ ]:
# Carga del archivo .csv con el corpus preprocesado y conversión de tipo de datos
df_corpus_procesado = pd.read_csv(f"{directorio_base}\data\corpus_procesado.csv")
df_corpus_procesado["id"] = df_corpus_procesado["id"].astype("string")
df_corpus_procesado["contenido"] = df_corpus_procesado["contenido"].astype("string")

In [ ]:
# Aplicar la vectorización a todo el corpus con las diferentes técnicas
print("Vectorizando corpus a BOW")
X_bow, bow_vectorizer = vectorize_bow(df_corpus_procesado["contenido_preproc_str"])
print("Vectorizando con TF-IDF")
X_tfidf, tfidf_vectorizer = vectorize_tfidf(df_corpus_procesado["contenido_preproc_str"])
print("Vectorizando con Word2Vec")
X_word2vec = vectorize_word2vec(df_corpus_procesado["contenido_preproc_str"], word2vec_model)

Vectorizando corpus a BOW
Vectorizando con TF-IDF
Vectorizando con Word2Vec


### Construcción de un índice invertido

Se implementó una función de nombre construir_indice_invertido, lo que hace es iterar sobre documento, y por cada token preprocesado que se encuentra en dicho documento crea un diccionario, por cada ocurrencia de ese token en ese documento o en otro se aumenta su valor de frecuencia en una unidad, es decir, se tendría un diccionario tipo:

- {Token_preprocesado01: {2203: 20, 2210: 15}}

Posteriormente, se guardarán estos resultados en un archivo de texto plano con la siguiente estructura:

- Término: Token_preprocesado01
    - Documento: 2203, Frecuencia: 20
    - Documento: 2210, Frecuencia: 15

Se creará una función para cargar el índice invertido desde este archivo de texto plano a un diccionario.

Nota: En el índice invertido se guardan únicamente los tokens preprocesados que tienen una longitud mayor o igual a tres, esto debido a que existen algunos con longitud dos que corresponden a abreviaciones que varían dependiendo al contexto del documento.

In [89]:
def construir_indice_invertido(df, directorio_base, texto_column='contenido_preproc_str', id_column='id'):
    """
    Construye un índice invertido a partir de un DataFrame con textos preprocesados.
    Solo almacena palabras con longitud >= 3 y ordena los términos por frecuencia descendente.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene los textos.
    - texto_column (str): Nombre de la columna que contiene los textos preprocesados.
    - id_column (str): Nombre de la columna que contiene los IDs de los documentos.
    
    Retorno:
    - indice_invertido (dict): Diccionario donde las claves son términos y los valores son diccionarios 
      con el ID del documento y la frecuencia del término.
    """
    # Estructura del índice invertido
    indice_invertido = defaultdict(lambda: defaultdict(int))
    
    # Construir el índice invertido
    for _, row in df.iterrows():
        doc_id = row[id_column]
        texto = row[texto_column]
        for token in texto.split():
            if len(token) >= 3:  # Filtrar palabras con longitud >= 3
                indice_invertido[token][doc_id] += 1  # Incrementar frecuencia
    
    # Ordenar términos por frecuencia descendente dentro de cada documento
    for termino in indice_invertido:
        indice_invertido[termino] = dict(sorted(indice_invertido[termino].items(), 
                                                key=lambda item: item[1], reverse=True))
        
    path_guardado_ii = f"{directorio_base}\data\indice_invertido.txt"
    guardar_indice_invertido(indice_invertido, path_guardado_ii)
    
    return f"Índice invertido creado y guardado exitosamente en data\indice_invertido.txt"


def guardar_indice_invertido(indice_invertido, output_path):
    """
    Guarda el índice invertido en un archivo de texto con formato legible.
    
    Parámetros:
    - indice_invertido (dict): Índice invertido a guardar.
    - output_path (str): Ruta del archivo de salida.
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for termino, documentos in sorted(indice_invertido.items()):  # Ordenar términos alfabéticamente
            f.write(f"Término: {termino}\n")
            for doc_id, frecuencia in documentos.items():  # Ya ordenados por frecuencia descendente
                f.write(f"    Documento: {doc_id}, Frecuencia: {frecuencia}\n")
            f.write("\n")  # Línea en blanco entre términos


def cargar_indice_invertido(input_path):
    """
    Carga el índice invertido desde un archivo de texto con formato legible.

    Parámetros:
    - input_path (str): Ruta del archivo de entrada.

    Retorno:
    - indice_invertido (dict): El índice invertido cargado desde el archivo.
    """
    indice_invertido = {}

    with open(input_path, 'r', encoding='utf-8') as f:
        termino = None
        for line in f:
            line = line.strip()
            if line.startswith("Término:"):
                termino = line.split("Término:")[1].strip()
                indice_invertido[termino] = {}
            elif line.startswith("Documento:"):
                doc_info = line.split(",")
                doc_id = doc_info[0].split("Documento:")[1].strip()
                frecuencia = int(doc_info[1].split("Frecuencia:")[1].strip())
                indice_invertido[termino][doc_id] = frecuencia
            # Ignorar líneas vacías
    return indice_invertido

In [92]:
construir_indice_invertido(df_corpus_procesado, directorio_base)

'Índice invertido creado y guardado exitosamente en data\\indice_invertido.txt'

In [91]:
indice_invertido = cargar_indice_invertido(f"{directorio_base}\data\indice_invertido.txt")

### Funciones para búsquedas

Se implementaron dos funciones de búsqueda, una que funcione para TF-IDF, BOW y WORD2VEC, utilizando distancia coseno como métrica para mostrar los resultados.

La segunda función por otro lado, es especializada para relizar búsqueda dentro del índice invertido construido en el paso anterior, esta función busca una única palabra, si se le proporciona una consulta con varias palabras y al preprocesarlas queda más de un token, se utilizará la primera válida. Si la palabran no existe dentro del índice invertido, dará un mensaje de error.

In [ ]:
def buscar_documentos(consulta, corpus_vectorizado, vectorizador, metodo="1"):
    """
    Busca documentos en el corpus y devuelve los más relevantes según el método elegido (BoW, TF-IDF, Word2Vec).
    """
    # Vectorizar la consulta
    if metodo == "1":
        consulta_vec = vectorizador.transform([consulta])
    elif metodo == "2":
        consulta_vec = vectorizador.transform([consulta])
    elif metodo == "3":
        consulta_vec = vectorize_word2vec([consulta], vectorizador)[0].reshape(1, -1)
    else:
        raise ValueError("Método no soportado")
    
    matriz = corpus_vectorizado

    # Calcular similitudes
    similitudes = cosine_similarity(consulta_vec, matriz)
    resultados = similitudes[0]
    
    # Rankear documentos por relevancia y tomar los top_n
    indices_ordenados = resultados.argsort()[::-1]
    
    return indices_ordenados, resultados

def buscar_palabra_indice_invertido(indice_invertido, df, termino, top_n=10, texto_column='contenido'):
    """
    Busca un término en el índice invertido y devuelve los 'top_n' documentos más relevantes.
    Solo muestra el contenido del documento.
    
    Parámetros:
    - indice_invertido (dict): El índice invertido construido.
    - df (pd.DataFrame): DataFrame que contiene los documentos con sus contenidos.
    - termino (str): El término a buscar en el índice invertido.
    - top_n (int): Número de resultados más relevantes a devolver.
    - texto_column (str): Columna que contiene el contenido del documento.
    
    Retorno:
    - lista_resultados (list): Lista de los documentos más relevantes con su contenido.
    """
    # Verificar si el término está en el índice invertido
    if termino in indice_invertido:
        # Obtener los documentos con el término y su frecuencia
        documentos = indice_invertido[termino]
        
        # Ordenar los documentos por frecuencia y tomar los 'top_n' más relevantes
        documentos_relevantes = sorted(documentos.items(), key=lambda item: item[1], reverse=True)[:top_n]
        #print(documentos_relevantes)
        
        # Extraer el contenido de los documentos relevantes
        lista_resultados = []
        for doc_id, _ in documentos_relevantes:
            # Obtener el contenido del documento desde el DataFrame
            contenido = df[df['id'] == doc_id][texto_column].values[0]
            lista_resultados.append([contenido, doc_id])
        
        return lista_resultados
    else:
        return []

### Ejemplos de usos prácticos

A continuación se muestra como debería ser el uso de todas las funciones ya implementadas anteriormente para realizar una simulación de búsqueda de una consulta dentro del corpus usando los diferentes métodos disponibles:

1. TF-IDF
2. BOW
3. Word2Vec
4. Índice invertido

Se devolverá el contenido de los documentos con los que exista una menor distancia respecto a la consulta, para el caso del índice invertido, se devolverá los documentos en orden descendente de acuerdo a la frecuencia del término en ese documento.

In [ ]:
# Definir el número máximo de resultados a mostrar, la consulta y preprocesar la consulta
max_resultados = 10
consulta = "problems with trading on united states of america"
consulta_preproc = preprocesar_contenido(consulta)

In [86]:
# ---- Paso 3.1: Ejemplo de búsqueda ifidf, bow, word2vec ----
resultados_tfidf = buscar_documentos(consulta_preproc, X_tfidf, tfidf_vectorizer, metodo="1")

# Mostrar los resultados
print("Resultados de la búsqueda (TF-IDF):")
# Realizar la busqueda y obtener resultados relevantes con search_functions
indices_ordenados, resultados = buscar_documentos(consulta_preproc, X_tfidf, tfidf_vectorizer, metodo="1")

# Preparar los resultados incluyendo índice, relevancia y texto
resultados_detallados = [
                {
                    "indice": indice,
                    "relevancia": resultados[indice],
                    "texto": df_corpus_procesado['contenido'][indice]  # Incluir el texto crudo del documento
                }
                for indice in indices_ordenados
            ]
print("")
print(f"Mostrando resultados relevantes para la búsqueda:   {consulta}")
print("")
for resultado in resultados_detallados[:max_resultados]:
    print(f"Índice: {resultado['indice']}, Relevancia: {resultado['relevancia']:.3f}")
    print(f"Texto: {resultado['texto']}\n")

Resultados de la búsqueda (TF-IDF):

Mostrando resultados relevantes para la búsqueda:   problems with trading on united states of america

Índice: 2842, Relevancia: 0.328
Texto: YEUTTER SAYS BUDGET CUT KEY TO BETTER U.S. TRADE
  A reduction of the U.S. federal budget
  deficit will be needed to help eliminate the nation's huge
  trade deficit, U.S. trade representative Clayton Yeutter said.
      Speaking to the New York Chamber of Commerce and Industry,
  Yeutter said "Capital and trade flows are clearly
  inter-releated now.
      "Unless we get the budget deficit down, we will not get the
  trade deficit down."
      He did not elaborate on his views of the linkages between
  the two deficits.
      Private analysts have said that the financing of large U.S.
  budget deficits requires heavy capital inflows from overseas
  investors through purchases of U.S. Treasury and, to a lesser
  extent, other U.S. securities as well.
      "We'll make some progress in reducing the 170 billion

In [ ]:
# ---- Paso 3.1: Ejemplo de búsqueda índice invertido ----

if len(consulta_preproc.split()) == 1:
    resultados = buscar_palabra_indice_invertido(indice_invertido, df_corpus_procesado, consulta_preproc, top_n=max_resultados)
elif len(consulta_preproc.split()) > 1:
    print("Búsqueda por índice invertido ha detectado más de una palabra, usando la primera válida...\n\n")
    resultados = buscar_palabra_indice_invertido(indice_invertido, df_corpus_procesado, consulta_preproc.split()[0], top_n=max_resultados)
else:
    print("La palabra es una stop-word o no se encuentra en el índice invertido...")
    pass

# Mostrar los resultados
for contenido, doc_id in resultados:
    print(f"Documento con ID: {doc_id}:\n{contenido}\n")


Búsqueda por índice invertido ha detectado más de una palabra, usando la primera válida...


Documento con ID: 144:
OPEC MAY HAVE TO MEET TO FIRM PRICES - ANALYSTS
  OPEC may be forced to meet before a
  scheduled June session to readdress its production cutting
  agreement if the organization wants to halt the current slide
  in oil prices, oil industry analysts said.
      "The movement to higher oil prices was never to be as easy
  as OPEC thought. They may need an emergency meeting to sort out
  the problems," said Daniel Yergin, director of Cambridge Energy
  Research Associates, CERA.
      Analysts and oil industry sources said the problem OPEC
  faces is excess oil supply in world oil markets.
      "OPEC's problem is not a price problem but a production
  issue and must be addressed in that way," said Paul Mlotok, oil
  analyst with Salomon Brothers Inc.
      He said the market's earlier optimism about OPEC and its
  ability to keep production under control have given way to 